In [ ]:
# from google.colab import drive
# drive.mount('MyDrive')

In [ ]:
!pip install 'keras<3.0.0' mediapipe-model-maker

In [ ]:
!pip install typeguard>=4.0.1

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vishnukv64/cvcclinicdb

In [ ]:
!unzip -q /content/cvcclinicdb.zip

replace CVC-ClinicDB/Ground_Truth/1.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
!pip install natsort

In [ ]:
import os
import natsort
import torch
from tqdm import tqdm
import numpy as np
import glob
from PIL import Image
import albumentations
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import helper
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
Data_dir_images = "CVC-ClinicDB/Original"
Data_dir_masks = "CVC-ClinicDB/Ground_Truth"


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
batch_size = 2

# class_names = ["polyp"]

images_list = glob.glob(f"{Data_dir_images}/*")
images_list = natsort.natsorted(images_list, reverse = False)
print(len(images_list))
masks_list = glob.glob(f"{Data_dir_masks}/*")
masks_list = natsort.natsorted(masks_list, reverse = False)
print(len(masks_list))

split=0.1
total_size = len(images_list)
valid_size = int(split * total_size)
test_size = int(split * total_size)

train_img, valid_img , train_masks, valid_masks = model_selection.train_test_split(images_list, masks_list, test_size=valid_size, random_state = 42)

train_img, test_img , train_masks, test_masks = model_selection.train_test_split(train_img, train_masks, test_size=test_size, random_state = 42)

train_img_tunner, valid_img_tunner , train_masks_tunner, valid_masks_tunner = model_selection.train_test_split(images_list, masks_list, test_size=valid_size, random_state = 42)

train_img_tunner, test_img_tunner , train_mask_tunners, test_masks_tunner = model_selection.train_test_split(train_img_tunner, train_masks_tunner, test_size=test_size, random_state = 42)

612
612


In [ ]:
def read_data_tunner(images, masks):
    final_images, final_masks = list(), list()
    for e, item in enumerate(images):
        image = cv2.imread(images[e], cv2.IMREAD_GRAYSCALE) # , cv2.IMREAD_GRAYSCALE#we use cv2 because PIL didnt load the image and we convert later to PIL image
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        final_images.append(image)

        mask = cv2.imread(masks[e], cv2.IMREAD_GRAYSCALE)#we use cv2 because PIL didnt load the image and we convert later to PIL image
        # mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        final_masks.append(mask)

    return np.array(final_images), np.array(final_masks)

In [ ]:
train_images_tunner, train_masks_tunner = read_data_tunner(train_img_tunner, train_masks_tunner)
valid_images_tunner, valid_masks_tunner = read_data_tunner(valid_img_tunner, valid_masks_tunner)
test_images_tunner, test_masks_tunner = read_data_tunner(test_img_tunner, test_masks_tunner)

In [ ]:
train_images_tunner.shape

(490, 288, 384)

In [ ]:
# converting all mask pixels other than 0 to 1
for e,mask in enumerate(train_masks_tunner):
    train_masks_tunner[e][train_masks_tunner[e]>0] = 1

for e,mask in enumerate(valid_masks_tunner):
    valid_masks_tunner[e][valid_masks_tunner[e]>0] = 1

for e,mask in enumerate(test_masks_tunner):
    test_masks_tunner[e][test_masks_tunner[e]>0] = 1

In [ ]:
# image size
SIZE_X = 384
SIZE_Y = 288
n_classes = 2  # class 1 is background, class 2 is of polyp

In [ ]:
# adding 3rd dimension for unet model
# normalizing image with keras normalizer
# it divides each pixel with 255 to change pixel between 0-1
#
train_images_tunner = np.expand_dims(train_images_tunner, axis=3)
# train_images = normalize(train_images, axis=1)

valid_images_tunner = np.expand_dims(valid_images_tunner, axis=3)
# valid_images = normalize(valid_images, axis=1)

test_images_tunner = np.expand_dims(test_images_tunner, axis=3)
# test_images = normalize(test_images, axis=1)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
train_masks_cat_tunner = to_categorical(train_masks_tunner, num_classes=n_classes)
test_masks_cat_tunner = to_categorical(test_masks_tunner, num_classes=n_classes)
valid_masks_cat_tunner = to_categorical(valid_masks_tunner, num_classes=n_classes)

In [ ]:
from tensorflow.keras.backend import cast
import tensorflow as tf

In [ ]:
def focalLoss(alpha=0.2, gamma=3, rate=0.1, weights=0, bias=0):
    def customloss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
  #     y_pred = tf.cast(y_pred, tf.float64)

        # bias = 0
        weighted_sum = tf.math.abs(tf.cast(weights*gamma + bias, tf.float32))
        # weighted_sum = tf.math.abs(tf.cast(weights*gamma + bias + gamma, tf.float32))
        #weighted_sum = tf.math.abs(tf.cast(weights + bias + gamma, tf.float32)) # algo 1
        y_predicted = tf.math.abs(tf.nn.sigmoid(weighted_sum))

        gamma_d = tf.math.reduce_mean(y_predicted-y_true)
        bias_d = tf.math.reduce_mean(y_predicted-y_true)

        gamma_ = tf.math.round(gamma - rate * gamma_d)

        ce = tf.nn.sigmoid_cross_entropy_with_logits(
                labels=y_true, logits=y_pred)
        alpha1 = y_true * alpha + (1. - y_true) * (1. - alpha)

        pt = tf.where(y_true==1, x=y_pred, y=1-y_pred)
        F_loss =  alpha1 * (1. - pt) ** gamma_ * ce
        return tf.reduce_mean(F_loss)
    return customloss

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import backend as K

def get_iou_vector(A, B):
    t = A>0
    p = B>0
    intersection = np.logical_and(t,p)
    union = np.logical_or(t,p)
    iou = (np.sum(intersection) + 1e-10 )/ (np.sum(union) + 1e-10)
    return iou


def check_units(y_true, y_pred):
    if y_pred.shape[1] != 1:
      y_pred = y_pred[:,1:2]
      y_true = y_true[:,1:2]
    return y_true, y_pred

def precision(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    y_true, y_pred = check_units(y_true, y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def iou_metric(label, pred):
    return tf.numpy_function(get_iou_vector, [label, pred>0.5], tf.float64)

In [ ]:
!pip install keras_tuner

In [ ]:
import keras_tuner

In [ ]:
from keras.layers import Input, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate
from keras.layers import Conv2D
from keras.models import Model

In [ ]:
# from keras.layers import Input, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate
# from keras.layers.convolutional import Conv2D
# from keras.models import Model

In [ ]:
def base_model(hp):
    in_layer = Input(shape=(None, None, 1), dtype='float32', name='input_1')

    c1 = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(in_layer)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D(pool_size=(2, 2),padding='valid',strides=(2, 2))(c1)

    c2 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D(pool_size=(2, 2),padding='valid',strides=(2, 2))(c2)

    c3 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D(pool_size=(2, 2),padding='valid',strides=(2, 2))(c3)

    c4 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D(pool_size=(2, 2),padding='valid',strides=(2, 2))(c4)

    c5 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c5)
    c5 = BatchNormalization()(c5)
    x5 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    x5 = concatenate([x5,c4], axis=-1)

    c6 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x5)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c6)
    c6 = BatchNormalization()(c6)
    x6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    x6 = concatenate([x6,c3], axis=-1)

    c7 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x6)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c7)
    c7 = BatchNormalization()(c7)
    x7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    x7 = concatenate([x7,c2], axis=-1)

    c8 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x7)
    c8 = BatchNormalization()(c8)
    c8 = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c8)
    c8 = BatchNormalization()(c8)
    x8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    x8 = concatenate([x8,c1], axis=-1)

    c9 = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(x8)
    c9 = BatchNormalization()(c9)
    c9 = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(c9)
    c9 = BatchNormalization()(c9)
    c10 = Conv2D(filters=2, kernel_size=(1, 1), strides=(1, 1), padding='valid', activation='softmax')(c9)

    model = Model(inputs=[in_layer], outputs=[c10])
    weights = K.sum(model.get_weights()[0])

    hp_gamma = hp.Int('gamma', min_value=1, max_value=9, step=1)
    adm = optimizers.Adam(learning_rate=1e-3)
    model.compile(loss=focalLoss(gamma=hp_gamma, weights=weights), optimizer=adm, metrics=['accuracy',iou_metric,
                                                                          metrics.Precision(), metrics.Recall()])
    return model

In [ ]:
tuner = keras_tuner.Hyperband(base_model,
                     objective='val_loss',
                     max_epochs=8,
                     factor=3,
                     directory='/content/Drive/MyDrive/unet_dir',
                     project_name='focal_unet_algo1')

In [ ]:
tuner.search(train_images_tunner,train_masks_cat_tunner,
             batch_size=8,
             epochs=20,
             validation_data=(valid_images_tunner,valid_masks_cat_tunner)
             )

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The  Optimal Gamma
is {best_hps.get('gamma')}.
""")

In [ ]:
best_gamma = best_hps['gamma']

print (f"best_gamma : {best_gamma}")

**Paper 2 code start from Here **

In [ ]:
def read_data(images, masks):
    final_images, final_masks = list(), list()
    for e, item in enumerate(images):
        image = cv2.imread(images[e]) # , cv2.IMREAD_GRAYSCALE#we use cv2 because PIL didnt load the image and we convert later to PIL image
        image = image/255.
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        final_images.append(image)

        mask = cv2.imread(masks[e], cv2.IMREAD_GRAYSCALE)#we use cv2 because PIL didnt load the image and we convert later to PIL image
        mask = mask/255.
        # mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        final_masks.append(mask)

    return np.array(final_images), np.array(final_masks)

In [ ]:
train_images, train_masks = read_data(train_img, train_masks)
valid_images, valid_masks = read_data(valid_img, valid_masks)
test_images, test_masks = read_data(test_img, test_masks)

In [ ]:
train_images.shape

(490, 288, 384, 3)

In [ ]:
# converting all mask pixels other than 0 to 1
for e,mask in enumerate(train_masks):
    train_masks[e][train_masks[e]>0] = 1

for e,mask in enumerate(valid_masks):
    valid_masks[e][valid_masks[e]>0] = 1

for e,mask in enumerate(test_masks):
    test_masks[e][test_masks[e]>0] = 1

In [ ]:
from keras.layers import Input, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate
from keras.layers import Conv2D
from keras.models import Model

In [ ]:
train_masks_cat = to_categorical(train_masks, num_classes=n_classes)
test_masks_cat = to_categorical(test_masks, num_classes=n_classes)
valid_masks_cat = to_categorical(valid_masks, num_classes=n_classes)

In [ ]:
!pip install tensorflow_addons

In [ ]:
# External libraries
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Conv1D, UpSampling2D
from tensorflow.keras.layers import concatenate, multiply, add, Activation
from tensorflow.keras.layers import Input, concatenate
# First, import the necessary module
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU

import tensorflow as tf
import math
import numpy as np

# Focus Gate
def FocusGate(input, skip_connections, n_filters, gamma=True, stats='mean', conv_transpose=False):
    """
    Parameters
    ----------
    input : gating signal
    skip_connections : incoming skip connection to be refined
    n_filters : number of filters
    gamma : whether to apply non-linear attention. Gamma is automatically tuned - see our latest paper
            for more details (https://ieeexplore.ieee.org/document/9761414)
    stats : either 'mean', 'max' or 'mean_max' to apply either global average pooling, global max pooling,
            or both
    conv_transpose : if false applies upsampling
    """

    # Resize input to match number of channels for skip connections
    resized_input = Conv2D(skip_connections.shape[-1], kernel_size=1, strides=(1, 1), padding='same', use_bias=True)(input)
    # Resize skip connections to match image shape for input
    resized_skip = Conv2D(skip_connections.shape[-1], kernel_size=1, strides=(2, 2), padding='same', use_bias=False)(skip_connections)

    stride_x = resized_skip.shape[1] // input.shape[1]
    stride_y = resized_skip.shape[2] // input.shape[2]

    if conv_transpose:
        resized_input = Conv2DTranspose(skip_connections.shape[-1], (stride_x, stride_y),strides=(stride_x, stride_y),padding='same')(resized_input)
    else:
        resized_input = UpSampling2D((stride_x,stride_y))(resized_input)

    # element wise addition
    added  = add([resized_input, resized_skip])

    # perform non-linear activation
    act = Activation('relu')(added)

    # channel attention
    channel_attention = channel_attention_block(act, stats)

    # spatial attention
    spatial_attention = spatial_attention_block(act, stats)

    # combine channel and spatial weights
    weights = multiply([channel_attention, spatial_attention])

    # focal parameter for tuneable background suppression
    if gamma:
        weights = Gamma()(weights)

    # rescale attention coefficient matrix to size of skip connection
    stride_x_weights = skip_connections.shape[1] // weights.shape[1]
    stride_y_weights = skip_connections.shape[2] // weights.shape[2]

    # Upsample to match original skip connection resolution
    if conv_transpose:
        weights = Conv2DTranspose(skip_connections.shape[-1], (stride_x_weights, stride_y_weights), strides=(stride_x_weights, stride_y_weights), padding='same')(weights)
    else:
        weights = UpSampling2D((stride_x_weights, stride_y_weights))(weights)

    # multiply skip connections by weights
    weights = multiply([weights, skip_connections])

    # perform final convolution and batch normalisation
    output = Conv2D(skip_connections.shape[-1], kernel_size=1, strides=(1, 1), padding='same', use_bias=True)(weights)
    output = BatchNormalization()(output)

    return output

# Automatically tuned gamma parameter
class Gamma(tf.keras.layers.Layer):

    def __init__(self):
        super(Gamma, self).__init__()

    def build(self, input_shape):
        initializer = tf.keras.initializers.Ones()

        self.w = self.add_weight(
            shape=(1,1),
            initializer=initializer,
            trainable=True,
            constraint= tf.keras.constraints.NonNeg())

    def call(self, inputs):
        inputs = tf.clip_by_value(inputs, tf.keras.backend.epsilon(), 1.)
        return (inputs)**self.w

def generate_block(input_layer, filters, dropout_rate=0.2):
    x = Conv2D(filters, (3, 3), padding='same', strides=1)(input_layer)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(filters, (3, 3), padding='same', strides=1)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    return x

# Gating signal
def GatingSignal(input, skip_connections):
    """
    Parameters
    ----------
    input : layer prior to upsampling
    skip_connections : incoming skip connection to be refined
    """
    signal = Conv2D(skip_connections.shape[-1], (1, 1), strides=(1, 1), padding="same")(input)
    signal = LeakyReLU(alpha=0.1)(signal)
    signal = BatchNormalization()(signal)
    return signal

def spatial_kernel_size(input, beta=1, gamma=2):
    """
    The original implementation based the kernel size on the channel, but with
    square images, it can be simplified to:
    k = | log2(R)/gamma + b/gamma|odd
    """

    k = int(abs(np.log2((input.shape[1])/gamma) + beta/gamma))
    out = k if k % 2 else k + 1

    return out

def channel_kernel_size(input, beta=1,gamma=2):
    """
    k = | log2(C)/gamma + b/gamma|odd
    """
    k = int(abs(np.log2(input.shape[-1]/gamma) + beta/gamma))
    out = k if k % 2 else k + 1
    return out

def spatial_attention_block(input, stats):
    """ Adaptive spatial attention block
    """

    k_size = spatial_kernel_size(input)

    # generate spatial statistics
    if (stats == 'mean') or (stats == 'mean_max'):
        mean = tf.reduce_mean(input, axis=-1, keepdims=True)
        t = mean

    if (stats == 'max') or (stats == 'mean_max'):
        maximum = tf.reduce_max(input, axis=-1, keepdims=True)
        t = maximum

    if (stats == 'mean_max'):
        t = concatenate([mean,maximum], axis=-1)

    t = Conv2D(1, kernel_size=k_size, padding="same",activation='sigmoid', use_bias = False)(t)

    return t

def channel_attention_block(input, stats):
    """ Adaptive channel attention block
    """

    k_size = channel_kernel_size(input)

    # generate channel statistics
    if (stats == 'mean') or (stats == 'mean_max'):
        mean = tf.reduce_mean(input, axis=[1,2], keepdims=True)
        mean = tf.squeeze(mean,axis=(-2))
        mean = tf.transpose(mean,perm=[0,2,1])
        t = mean

    if (stats == 'max') or (stats == 'mean_max'):
        maximum = tf.reduce_max(input, axis=[1,2], keepdims=True)
        maximum = tf.squeeze(maximum,axis=(-2))
        maximum = tf.transpose(maximum,perm=[0,2,1])
        t = maximum

    if (stats == 'mean_max'):
        t = concatenate([mean,maximum], axis=-1)

    t = Conv1D(filters=1, kernel_size=k_size, padding='same', use_bias=False)(t)
    t = tf.transpose(t,perm=[0,2,1])
    t = tf.expand_dims(t,(-2))
    t = tf.math.sigmoid(t)

    return t

# def conv_layer_2D(input, neurons):

#     conv1 = Conv2D(neurons, (3, 3), padding='same', activation='relu', strides=1)(input)
#     conv1 = BatchNormalization()(conv1)
#     conv2 = Conv2D(neurons, (3, 3), padding='same', activation='relu', strides=1)(conv1)
#     out = BatchNormalization()(conv2)

#     return out

    # Then, replace BatchNormalization with InstanceNormalization in the conv_layer_2D function
def conv_layer_2D(input, neurons, dropout_rate=0.2):
    conv1 = Conv2D(neurons, (3, 3), padding='same', strides=1)(input)
    conv1 = LeakyReLU(alpha=0.1)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(dropout_rate)(conv1)
    conv2 = Conv2D(neurons, (3, 3), padding='same', strides=1)(conv1)
    conv2 = LeakyReLU(alpha=0.1)(conv2)
    out = tfa.layers.InstanceNormalization()(conv2)
    out = Dropout(dropout_rate)(out)
    conv3 = Conv2D(neurons, (3, 3), padding='same', strides=1)(out)
    conv3 = LeakyReLU(alpha=0.1)(conv3)
    out = BatchNormalization()(conv3)
    out = Dropout(dropout_rate)(out)
    return out


# U-Net model with MobileNetV2 backbone for comparison

base_model = tf.keras.applications.MobileNetV2(input_shape=[288,384, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

def unet_model(output_channels:int):
    inputs = tf.keras.layers.Input(shape=[288,384, 3])

    # Downsampling through the model
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
      x = up(x)
      concat = tf.keras.layers.Concatenate()
      x = concat([x, skip])
      x = generate_block(x, output_channels)
    # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
        filters=output_channels, kernel_size=3, strides=2,
        padding='same')  #64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

# Focus U-Net model with MobileNetV2 backbone
def focus_unet_model(output_channels:int):
    feature_map = [32, 64, 128, 256, 512]
    inputs = tf.keras.layers.Input(shape=[288,384, 3])
    contracting_convs = down_stack(inputs)
    cnn_chain = contracting_convs[-1]
    contracting_convs = reversed(contracting_convs[:-1])
    for skip_connection, filters in zip(contracting_convs, reversed(feature_map[:-1])):
        cnn_chain = Conv2DTranspose(filters, (2, 2), strides=(2, 2),padding='same')(cnn_chain)
        cnn_chain = concatenate([cnn_chain, skip_connection], axis=-1)
        cnn_chain = conv_layer_2D(cnn_chain, filters)
        cnn_chain = Conv2D(filters, (3, 3), padding='same', strides=1)(cnn_chain)
        cnn_chain = LeakyReLU(alpha=0.1)(cnn_chain)
        cnn_chain = BatchNormalization()(cnn_chain)
    last = tf.keras.layers.Conv2DTranspose(
        filters=output_channels, kernel_size=3, strides=2,
        padding='same')
    x = last(cnn_chain)
    return tf.keras.Model(inputs=inputs, outputs=x)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


9406464/9406464 [==============================] - 2s 0us/step


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
model = focus_unet_model(2)
adm = Adam(learning_rate=1e-3)
model.compile(loss=focalLoss(gamma=best_gamma), optimizer=adm, metrics=['accuracy',iou_metric])

In [ ]:
model.fit(train_images,train_masks_cat,
             batch_size=4,
             epochs=12,
             validation_data=(valid_images,valid_masks_cat))

In [ ]:
test_predictions = model.predict(test_images)
test_pred_classes = np.argmax(test_predictions, axis=-1)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from tensorflow.keras.metrics import MeanIoU

# Flatten the arrays
true = test_masks.flatten()
pred = test_pred_classes.flatten()

# Compute metrics
accuracy = accuracy_score(true, pred)
f1 = f1_score(true, pred)
precision = precision_score(true, pred)
recall = recall_score(true, pred)
auc = roc_auc_score(true, pred)

# For IoU, we can use tf.keras.metrics
m = MeanIoU(num_classes=2)
m.update_state(true, pred)
iou = m.result().numpy()

print('Accuracy: ', accuracy)
print('F1 score: ', f1)
print('Precision: ', precision)
print('Recall: ', recall)
print('AUC: ', auc)
print('IoU: ', iou)



In [ ]:
test_predictions = model.predict(test_images)
test_pred_classes = np.argmax(test_predictions, axis=-1)

# Flatten the arrays
t = test_masks.flatten()
p = test_pred_classes.flatten()

2/2 [==============================] - 0s 236ms/step


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

predict = model.predict(test_images)
# Flatten the arrays
t = test_masks.flatten()
p = test_pred_classes.flatten()

def report(y_true, y_pred):
    print('\n        ----------Test Data------------------')
    print('Accuracy: {:,.2f}'.format(accuracy_score(y_true, y_pred )* 100))
    print('Precision: {:,.2f}'.format(precision_score(y_true, y_pred, average='macro') * 100))
    print('Recall-score: {:,.2f}'.format(recall_score(y_true, y_pred, average='macro') * 100))
    print('F1-score: {:,.2f}'.format(f1_score(y_true, y_pred, average='macro') * 100))
    print('AUC-score: {:,.2f}'.format(roc_auc_score(y_true, y_pred) * 100))
    # For IoU, we can use tf.keras.metrics
    m = MeanIoU(num_classes=2)
    m.update_state(y_true, y_pred)
    iou = m.result().numpy()
    print('IoU-score: {:,.2f}'.format(iou*100))

report(t,p)


# Accuracy: 98.12
# Precision: 94.38
# Recall-score: 93.88
# F1-score: 94.13
# AUC-score: 93.88
# IoU-score: 97.31